There is a decent number of strange matches in the dataset.  Here, we consider including or excluding various kinds of strange matches both from training and from evaluation.  Our hope is that we can create a meaningful improvement in model performance through manipulating these filters.

In [1]:
from tennis_new.fetch.tennis_explorer.combiner import read_joined

jd = read_joined()

/Users/siddhantjagadish/Documents/DataProjects/tennis_new/venv_377/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Now let's modify our metrics to allow for nuanced understanding of the possible filtering we want to do.  Specifically, we want to examine the effects of including / excluding:

* Retirements: Identified as cases where the winner won 1 set, and p1_set1 is not null
* Walkovers: Identified as cases where the winner won only 1 set, and p1_set1 is null
    * Some of these cases have errors in recording -- partial scores for retirements were not recorded on TennisExplorer.  One example of this kind of case is [/match-detail/?id=14145](http://www.tennislive.net/atp/match/richey-reneberg-VS-oscar-burrieza-lopez/australian-open-melbourne-1997/)
        
Let's start by getting our baseline performance

In [3]:
from tennis_new.model.config.elo.global_set_elo import SetELO

set_elo = SetELO()
set_elo.run(jd)
set_elo.validation_evaluation

{'DummyFilter_prediction_AUCMetric': 0.8187031847302881,
 'DummyFilter_prediction_AccuracyMetric': 0.7358520800135314,
 'DummyFilter_prediction_LogLikelihoodMetric': -0.5226366377611569,
 'HasOddsFilter_prediction_AUCMetric': 0.7839029874196454,
 'HasOddsFilter_prediction_AccuracyMetric': 0.7056423354253945,
 'HasOddsFilter_prediction_LogLikelihoodMetric': -0.5594758958654537,
 'DummyFilter_odds_implied_probability_AUCMetric': None,
 'DummyFilter_odds_implied_probability_AccuracyMetric': None,
 'DummyFilter_odds_implied_probability_LogLikelihoodMetric': None,
 'HasOddsFilter_odds_implied_probability_AUCMetric': 0.7937506478103871,
 'HasOddsFilter_odds_implied_probability_AccuracyMetric': 0.7114980299325661,
 'HasOddsFilter_odds_implied_probability_LogLikelihoodMetric': -0.5501844612492598}

Now let's subclass our model config with a new evaluator to get a more nuanced view of what's happening:

In [4]:
from tennis_new.model.utils import filters
from tennis_new.model.utils.evaluation import Evaluator
from tennis_new.model.utils.metrics import AccuracyMetric, AUCMetric 


class HasOddsNoWalkFilter(filters.Filter):
    sub_filters = [
        filters.HasOddsFilter(),
        filters.PossibleWalkoverFilter()
    ]

    
class CompleteMatchFilter(filters.Filter):
    sub_filters = [
        filters.PossibleWalkoverFilter(),
        filters.RetirementFilter()
    ]

    
class HasOddsCompleteMatchFilter(filters.Filter):
    sub_filters = [
        filters.HasOddsFilter(),
        CompleteMatchFilter()
    ]

    
class InvestigationEvaluator(Evaluator):
    
    @property
    def filters(self):
        return [
            filters.DummyFilter(),
            filters.PossibleWalkoverFilter(),
            CompleteMatchFilter(),
            filters.HasOddsFilter(),
            HasOddsNoWalkFilter(),
            HasOddsCompleteMatchFilter()
        ]

    @property
    def metrics(self):
        return [
            AccuracyMetric(),
            AUCMetric()
        ]
    

class InvestigationSetELO(SetELO):

    @property
    def evaluator(self):
        return InvestigationEvaluator('prediction', self.baseline_pred_col)
    

baseline_model = InvestigationSetELO()
baseline_model.run(jd)
baseline_model.validation_evaluation

{'DummyFilter_prediction_AccuracyMetric': 0.7358520800135314,
 'DummyFilter_prediction_AUCMetric': 0.8187031847302881,
 'PossibleWalkoverFilter_prediction_AccuracyMetric': 0.737773727542894,
 'PossibleWalkoverFilter_prediction_AUCMetric': 0.8211988276191851,
 'CompleteMatchFilter_prediction_AccuracyMetric': 0.7433517659154695,
 'CompleteMatchFilter_prediction_AUCMetric': 0.8276610280137657,
 'HasOddsFilter_prediction_AccuracyMetric': 0.7056423354253945,
 'HasOddsFilter_prediction_AUCMetric': 0.7839029874196454,
 'HasOddsNoWalkFilter_prediction_AccuracyMetric': 0.7062759936208136,
 'HasOddsNoWalkFilter_prediction_AUCMetric': 0.7848521712141644,
 'HasOddsCompleteMatchFilter_prediction_AccuracyMetric': 0.7124238101413565,
 'HasOddsCompleteMatchFilter_prediction_AUCMetric': 0.7924853974658239,
 'DummyFilter_odds_implied_probability_AccuracyMetric': None,
 'DummyFilter_odds_implied_probability_AUCMetric': None,
 'PossibleWalkoverFilter_odds_implied_probability_AccuracyMetric': None,
 'Possi

Now let's try removing the walkovers from training and see what happens 

In [5]:
class NoWalkoverTrainingFilter(filters.Filter):
    
    sub_filters = filters.TrainingFilter.sub_filters + [filters.PossibleWalkoverFilter()]
    
class SetELONoWalkover(InvestigationSetELO):
    
    @property
    def training_filter(self):
        return NoWalkoverTrainingFilter()
        
no_walkover_model = SetELONoWalkover()
no_walkover_model.run(jd)
no_walkover_model.validation_evaluation

{'DummyFilter_prediction_AccuracyMetric': 0.7353669067310004,
 'DummyFilter_prediction_AUCMetric': 0.8184461581251941,
 'PossibleWalkoverFilter_prediction_AccuracyMetric': 0.7372754289408237,
 'PossibleWalkoverFilter_prediction_AUCMetric': 0.8209598815326888,
 'CompleteMatchFilter_prediction_AccuracyMetric': 0.7428284287844684,
 'CompleteMatchFilter_prediction_AUCMetric': 0.8274457821801312,
 'HasOddsFilter_prediction_AccuracyMetric': 0.705346435968915,
 'HasOddsFilter_prediction_AUCMetric': 0.7839522594090187,
 'HasOddsNoWalkFilter_prediction_AccuracyMetric': 0.7059632884080178,
 'HasOddsNoWalkFilter_prediction_AUCMetric': 0.784908206161682,
 'HasOddsCompleteMatchFilter_prediction_AccuracyMetric': 0.71205096615225,
 'HasOddsCompleteMatchFilter_prediction_AUCMetric': 0.792566927985133,
 'DummyFilter_odds_implied_probability_AccuracyMetric': None,
 'DummyFilter_odds_implied_probability_AUCMetric': None,
 'PossibleWalkoverFilter_odds_implied_probability_AccuracyMetric': None,
 'PossibleW

In general, removing the "possible walkovers" from training doesn't seem to help my results much (it even hurts them slightly), even when removing the possible walkovers from evaluation.  Let's try filtering out the retirements, just for shits and giggles. 

In [6]:
class NoRetirementTrainingFilter(filters.Filter):
    
    sub_filters = filters.TrainingFilter.sub_filters + [filters.RetirementFilter()]
    
class SetELONoRetirement(InvestigationSetELO):
    
    @property
    def training_filter(self):
        return NoRetirementTrainingFilter()
        
no_retirement_model = SetELONoRetirement()
no_retirement_model.run(jd)
no_retirement_model.validation_evaluation

{'DummyFilter_prediction_AccuracyMetric': 0.7347259438623355,
 'DummyFilter_prediction_AUCMetric': 0.817629042898023,
 'PossibleWalkoverFilter_prediction_AccuracyMetric': 0.7366244983345155,
 'PossibleWalkoverFilter_prediction_AUCMetric': 0.8201479895566363,
 'CompleteMatchFilter_prediction_AccuracyMetric': 0.7423606672779985,
 'CompleteMatchFilter_prediction_AUCMetric': 0.8267733311166058,
 'HasOddsFilter_prediction_AccuracyMetric': 0.7049415209232063,
 'HasOddsFilter_prediction_AUCMetric': 0.7831434638666345,
 'HasOddsNoWalkFilter_prediction_AccuracyMetric': 0.7055724068920229,
 'HasOddsNoWalkFilter_prediction_AUCMetric': 0.7841042266021381,
 'HasOddsCompleteMatchFilter_prediction_AccuracyMetric': 0.712002334327584,
 'HasOddsCompleteMatchFilter_prediction_AUCMetric': 0.7919668183008487,
 'DummyFilter_odds_implied_probability_AccuracyMetric': None,
 'DummyFilter_odds_implied_probability_AUCMetric': None,
 'PossibleWalkoverFilter_odds_implied_probability_AccuracyMetric': None,
 'Possib

This is even worse!  In conclusion, we will apply neither additional filter to model training.